In [1]:
import pandas as pd
import subprocess
import os
import time
import csv

import sys
sys.path.append('../GOCAM_Project/dev')
import utils

In [2]:
pd.options.display.max_colwidth = 100

In [3]:
identifiers = pd.read_csv('../data/identifiers.csv')
identifiers = identifiers['identifier']
identifiers= identifiers[identifiers.str.contains('reacto.owl')]
identifiers.reset_index(drop=True, inplace=True)

In [4]:
def query(rIDs,uIDs,sparql_file,o_file, members2setID, setID2members):
    #rIDs = [rID,crID]
    #FORMAT QUERY VALUES STRING
    queryIDs = rIDs['crID']
    queryIDs = queryIDs.drop_duplicates()
    queryIDs = queryIDs.apply(lambda x: '<'+x+'>') #convert to IRI
    queryIDs = queryIDs.to_string(index=False)
    
        
    #RUN QUERY WITH ROBOT ON REACTO.owl
    # '&' is used for sed, not '/' as a delimiter because the identifier contains '/'
    with open(o_file, "w") as outfile:
        subprocess.run(['sed',"s&REPLACETHIS&"+queryIDs.encode("unicode_escape").decode("utf-8")+"&",sparql_file], stdout=outfile)
    subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', o_file,'../temp/results.csv'])
    
    #log_results.append(pd.read_csv('../temp/results.csv'))
    results = handle_sets('../temp/results.csv', uIDs, members2setID, setID2members)
    #log_results.append(results)
    #RESULTS ONLY HAS crID, ADD rID column
    results = map_results_to_rID(rIDs,results)
    #log_results.append(results)
    return results

In [9]:
def f_recursive(rIDs,uIDs,depth, members2setID, setID2members):
    #QUERY
    if depth > 10:
        #Possible error
        return 1  #change to return the traceback instead
    results = query(rIDs,uIDs,'../query/query_for_sed-test.txt','../temp/query_for_robot.txt',  members2setID, setID2members )
    
    #SORT RESULTS
    p_list = results[results['type']== 'BioPAX type: interface org.biopax.paxtools.model.level3.Protein']
    c_list = results[results['type']== 'BioPAX type: interface org.biopax.paxtools.model.level3.Complex']

    #UPDATE uID DICTIONARY
    p_list['record'] = p_list['record'].apply(lambda x: x.split('/')[-1]) #just keep the uniprot ID
    uIDs = utils.update_dict(uIDs,p_list,'rID','record')
    
    
    #TERMINATING CONDITION (no more components to search)
    #missing = c_list[c_list['component'].isna()]
    #log_c.append(missing)
    log.append(c_list)
    c_list = c_list.dropna() #see http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1475437 as an example
    if len(c_list.index) == 0:
        return 
    
    #UPDATE rIDs FOR NEXT QUERY
    new_rIDs = c_list[['rID','component']]
    #new_rIDs.dropna(inplace=True) #only keep rows with a value for component
    new_rIDs = new_rIDs.rename(columns={'component':'crID'})
    #log_recursion.append(new_rIDs)
    #RECURSE
    depth = depth+1
    f_recursive(new_rIDs,uIDs, depth,  members2setID, setID2members)
    #how to handle reactome sets? (rSet)
    return  #returns a set

test_list = ['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1168601',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602359',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602468',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158197',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5359005',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-9606895',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-3238209'
            ]
test_list

In [ ]:
#len(uIDs.keys())

In [ ]:
#len(rIDs.drop_duplicates().rID)

In [ ]:
#set(rIDs.drop_duplicates().rID) - set(uIDs.keys()) 

In [ ]:
#set(uIDs.keys()) - set(rIDs.drop_duplicates().rID)

In [ ]:
#utils.dict2csv(uIDs,'../data/uIDs_test2.csv')
#utils.dict2csv(members2setID,'../data/members2setID_test2.csv')
#utils.dict2csv(setID2members,'../data/setID2members_test2.csv')

In [ ]:
#%%time
#subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', '../temp/query_for_robot-Comp.txt','../temp/comp.csv'])
#subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', '../temp/query_for_robot-Prot.txt','../temp/results.csv'])
#subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', '../temp/query_for_robot-Set.txt','../temp/set.csv'])


Try it two ways. first time, remove sets of proteins by running the not exist filter for sets. Then, try it with no filter and removing any rID with multiple entries for their record. Compare the two to see if there is any difference.

In [5]:
prots = pd.read_csv('../temp/prot.csv')
sets = pd.read_csv('../temp/set.csv')
comps = pd.read_csv('../temp/comp.csv')
len(prots),len(prots.crID.unique()),len(comps),len(comps.crID.unique())

(28520, 28520, 33560, 14125)

In [6]:
sets.type.value_counts()

BioPAX type: interface org.biopax.paxtools.model.level3.Protein           21870
BioPAX type: interface org.biopax.paxtools.model.level3.PhysicalEntity     5263
BioPAX type: interface org.biopax.paxtools.model.level3.Complex            3540
BioPAX type: interface org.biopax.paxtools.model.level3.SmallMolecule      1761
BioPAX type: interface org.biopax.paxtools.model.level3.Dna                 725
BioPAX type: interface org.biopax.paxtools.model.level3.Rna                 323
Isa http://purl.obolibrary.org/obo/PR_000027744                               2
Isa http://purl.obolibrary.org/obo/PR_000000031                               2
Isa http://purl.obolibrary.org/obo/PR_000027746                               2
Name: type, dtype: int64

In [7]:
sets = sets.loc[sets.type.apply(lambda x:(x == "BioPAX type: interface org.biopax.paxtools.model.level3.Complex") or (x == "BioPAX type: interface org.biopax.paxtools.model.level3.PhysicalEntity")or (x == "BioPAX type: interface org.biopax.paxtools.model.level3.Protein"))]
len(sets),len(sets.crID.unique())

(30673, 4809)

In [8]:
set(sets.crID.values).intersection(set(prots.crID.values)),set(sets.crID.values).intersection(set(comps.crID.values)),set(comps.crID.values).intersection(set(prots.crID.values))

(set(), set(), set())

In [9]:
loc = {k:'prots' for k in prots.crID}
d2 = {k:'comps' for k in comps.crID}
d3 = {k:'sets' for k in sets.crID}
loc.update(d2)
loc.update(d3)

In [10]:
len(loc)

47454

In [12]:
comps

,crID,name,component
0,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602363,PLA2G2A:Ca2+,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602364
1,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602363,PLA2G2A:Ca2+,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-ALL-74112
2,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-167127,HIV-1 transcription complex with (ser5) phosphorylated CTD containing extruded transcript to +30,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HIV-167131
3,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-167127,HIV-1 transcription complex with (ser5) phosphorylated CTD containing extruded transcript to +30,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-109634
4,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-167127,HIV-1 transcription complex with (ser5) phosphorylated CTD containing extruded transcript to +30,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-113404
...,...,...,...
33555,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173713
33556,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173671
33557,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173706
33558,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173717


In [34]:
sets

,crID,name,type,entity_from_set
0,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938258
1,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938259
2,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938261
3,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938262
4,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938272
...,...,...,...,...
33483,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268660,Procollagen type V,BioPAX type: interface org.biopax.paxtools.model.level3.Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268645
33484,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268660,Procollagen type V,BioPAX type: interface org.biopax.paxtools.model.level3.Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268659
33485,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268660,Procollagen type V,BioPAX type: interface org.biopax.paxtools.model.level3.Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268712
33486,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8854614,RAB33,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8854592


In [75]:
s1 = {'a','b'}
s2 = {'a','c'}
entities = [s1,s2]
u = set.intersection(*entities)
specifics = []
for entity in entities:
    specifics.append(entity - u)
specifics
sset = set.union(*specifics)
df = pd.DataFrame({'record':list(sset),'set_ID':'sset:'+'NAME'})
df

,record,set_ID
0,c,sset:NAME
1,b,sset:NAME


In [111]:
def recurse(rID,x,members2setID,setID2members,log):
    #log.append('searching for:'+x)
    if loc.get(x) == 'prots':
        #log.append('prot:'+x)
        result = prots.query('crID == @x')
        return list(set(result.record.values))
    elif loc.get(x) == 'comps':
        #log.append('comp:'+x)
        result = list(comps.query('crID == @x').component.apply(lambda y: recurse(rID, y, members2setID,setID2members, log))) 
        flat_list = list(set([item for sublist in result for item in sublist])) #once a complex is flattened, eliminate redundant entries (ie dimers)
        return flat_list
    elif loc.get(x) == 'sets':
        #log.append('set:'+x)
        result = sets.query('crID == @x')
        set_name  = result.name.values[0]
        
        result = list(sets.query('crID == @x').entity_from_set.apply(lambda y: recurse(rID, y,  members2setID,setID2members, log)))
        entities = [] 
        
        for entity in result:
            entities.append(set(entity))
            
        u = set.intersection(*entities)
        specifics = []
        for entity in entities:
            specifics.append(entity - u)
            if len(entity - u) > 1:
                log.append(rID + ' ' + crID)
        sset = set.union(*specifics)
        setID2members['sset:'+set_name] = sset
        df = pd.DataFrame({'set_ID':'sset:'+set_name,'record':list(sset)})
        result = ['sset:'+set_name]
     
        for i in u:
            result.append(i)
        members2setID = utils.update_dict(members2setID,df,'record','set_ID') #dict of {uniprot records:{set_IDs}}
        setID2members = utils.update_dict(setID2members,df,'set_ID','record') #dict of {set_ID:{uniprot records}}
        #log.append(result)
        return result
        

    else:
        log.append('not found:'+x)


In [112]:
#rIDs = identifiers.to_frame(name='rID')
rIDs = pd.DataFrame({'rID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1250351']})
rIDs['crID']=rIDs.rID
#rIDs=rIDs[3505:3506]
#rIDs=rIDs[rIDs['rID']=='http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158197'] #nested complex
#rIDs=rIDs[rIDs['rID']=='http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1168601']
#rIDs = pd.DataFrame({'rID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158172'],'crID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158172']})
rIDs = rIDs.drop_duplicates()
uIDs = {}
members2setID = {} 
setID2members = {}
log = []
rIDs['uIDs']=rIDs.crID.apply(lambda x: recurse(x,x,members2setID,setID2members,log))
rIDs.uIDs.values[0]

['sset:p-ERBB4cyt1 homodimers', 'sset:NRGs/EGF-like ligands']

In [108]:
setID2members.get('sset:NRGs/EGF-like ligands')

{'sset:EGF-like ligands', 'sset:NRG1/2/3/4'}

In [110]:
setID2members.get('sset:NRG1/2/3/4')

{'http://identifiers.org/uniprot/O14511',
 'http://identifiers.org/uniprot/P56975',
 'http://identifiers.org/uniprot/Q02297',
 'http://identifiers.org/uniprot/Q8WWG1'}

In [30]:
loc.get('http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158213')

'prots'

In [35]:
sets.query('crID == "http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273"')

,crID,name,type,entity_from_set
0,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938258
1,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938259
2,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938261
3,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938262
4,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938272
5,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938277
6,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938279


In [105]:
rIDs = identifiers.to_frame(name='rID')
rIDs['crID']=rIDs.rID
rIDs=rIDs[3505:3506]
rIDs

,rID,crID
3505,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5672720,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5672720


In [ ]:
%%time

log_c=[] #complex with no component
log=[] #c_list
log_rIDs = []
uIDs = {}
members2setID= {}
setID2members= {}
#df rID (the original rID from a GOCAM), crID (the current rID to search with)
#the first rIDs df has the original rIDs duplicated as the component to start the search
rIDs = identifiers.to_frame(name='rID')
rIDs['crID']=rIDs.rID
rIDs=rIDs[3505:3506]
rIDs = rIDs.drop_duplicates()

f_recursive(rIDs,uIDs,0, members2setID, setID2members)
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5672713' in uIDs
#utils.dict2csv(uIDs,'../data/uIDs.csv')
#utils.dict2csv(members2setID,'../data/members2setID.csv')
#utils.dict2csv(setID2members,'../data/setID2members.csv')